#  RNN (rekurentne neuralne mreže)

**"A class of neural networks with loops in them, allowing information to persist"**



## Pregled
* RNN
* Motivacija
* Vanila RNN
* LSTM
* Primeri
* Resursi
* Implementacija

## Rekurentne neuralne mreže (RNN)
* Tip NN koji se često koristi kada su ulazni podaci vremenske sekvence tj. kada je umesto **jednog** feature vektora ulaz **sekvenca** feature vektora kroz vreme pri čemu postoje veze između sadašnjosti i prošlosti
* **Primeri**: tekst, zvuk, video, genomi, rukopis, berza...
* Prva mreža koja nije feedforward, sadrži cikluse
* Novi metod treniranja: **backpropagation through time** (BPTT)
* Nekoliko fiksnih arhitektura

## Motivacija
* Nije jasno kako bi standardna NN "uhvatila" pravilnosti kroz vreme
* Kao primer posmatrajmo problem predikcije naredne reči: za ovo je potreban kontekst, tj. nemoguće je izvršiti predikciju na osnovu jedne prethodne reči

## Vanila RNN
* Najjednostavniji tip RNN
* Prva ilustracija sa [colah:Understanding LSTM Networks](http://colah.github.io/posts/2015-08-Understanding-LSTMs/): rekurentna veza (grana = težine tj. FC sloj)
* Kako bismo razumeli značenje ove rekurentne veze posmatramo **unrolled** prikaz (druga ilustracija na istom linku) - "odmotali" smo mrežu za fiksan broj vremenskih jedinica
* Sada stvari deluju jasnije: mreža ima svoje **skriveno stanje** koje zadržava kroz vreme i u svakom koraku ga obogaćuje novim ulazom i daje novi izlaz
* To skriveno stanje treba da "zapamti" prošlost i iskoristi je za dati zadatak
* U opštem slučaju u svakom vremenskom koraku imamo novi ulaz i novi izlaz, ali to ne mora biti slučaj (npr. u klasifikaciji zvuka važan nam je samo poslednji izlaz)
* [Još jedna korisna ilustracija koja prikazuje pojedinačne neurone](https://i.imgur.com/yF92R2g.png)
* Na novoj ilustraciji imamo označene tri matrice težina: 
  * $W_{ih}$ - matrica input->hidden, na slici označena kao w1
  * $W_{hh}$ - matrica hidden->hidden, na slici označena kao w2
  * $W_{ho}$ - matrica hidden->output, na slici označena kao w3
* Pored toga ćemo imati dve funkcije aktivacije i dve bias vrednosti:
   * $f_h$ - funkcija aktivacije skrivenog sloja (npr. tanh)
   * $b_h$ - bias skrivenog sloja
   * $f_o$ - funkcija aktivacije izlaznog sloja (npr. sigmoid)
   * $b_o$ - bias izlaznog sloja
* Kao i do sada ulazni vektor označavamo sa $X$, a izlazni sa $Y$
* Skriveno stanje označavamo sa $H$
* Sada su kompletne formule za forward propagation **u jednom vremenskom trenutku** kojima od ulaza i starog skrivenog stanja dobijamo izlaz i novo skriveno stanje $(X, H) \to (H, Y)$:
  * $H_{new} = f_h(W_{ih} \cdot X + W_{hh} \cdot H + b_h) $
  * $Y = f_o(W_{ho} \cdot H + b_o) $
* Što se tiče treninga tj. backpropagation uvodimo metod **backpropagation through time** (BPTT)
  * BPTT koristi identične principe kao standardni BP, tj. propagira gradijente unazad od troška i nije suštinski drukčiji - novo ime koristimo jer je više deskriptivno
* Postoji i **bidirectional RNN** varijanta u kojoj postoje dva skrivena stanja: jedno se propagira kroz vreme unazad a jedno unapred
* **Problem sa vanilla RNN**: ne uspevaju da uhvate pravilnosti kada su sekvence dugačke
  - Jedan od konkretnih problema: u slučaju jako dugih sekvenci BPTT stepenuje matricu $W_{hh}$ dovoljno puta da se gradijenti koji služe za ažuriranje težina izgube (vanishing gradients) ili odu u beskonačno (exploding gradients)
* Zbog ovoga se umesto "klasičnih" RNN koriste naprednije arhitekture
* Najpopularnija takva arhitektura je LSTM 
  
## LSTM (Long Short-Term Memory)
* Pratiti [ilustracije](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) u sekciji *LSTM Networks*
* Umesto jednostavnog pravila za forward propagation koristimo kompleksne **LSTM ćelije**
* Jedna ćelija se sadrži od tri kapije: input gate, forget gate, output gate
* TODO jednačine
* Pored LSTM često se koristi i GRU (Gated Recurrent Unit)

## Primeri
* [Modeli za predikciju narednog karaktera - generisanje teksta](http://karpathy.github.io/assets/rnn/charseq.jpeg): Šekspir, Wikipedia, Latex, Linux sors kod
* [Još primera korišćenja](https://medium.com/datathings/the-magic-of-lstm-neural-networks-6775e8b540cd): prepoznavanje rukopisa, generisanje rukopisa, generisanje muzike, prevođenje, image captioning...
* Naredni korak: attention?

## Resursi
- [colah: uvod u RNN fokusiran na LSTM, odlične ilustracije](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)
- [karpathy: char-rnn, odličan primer primene](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)
- [Implementacija u numpy](https://github.com/rand0musername/psiml2017-workshops/blob/master/4%20-%20RNN/rnn_fwd.py)

## Implementacija RNN (LSTM) u Tensorflow

In [0]:
# TODO